## Datenvorverarbeitung
Dieser Pythonskript entpricht Abschnitt 4.3 der Ausarbeitung.
Bevor lernbasierte oder lexikalische Ansätze zur Sentimentanalyse angewendet werden können, müssen die Textdaten zunächst vorverarbeitet werden.
Ziel ist es, dass das Textfeld nur Wörter enthält.

In [1]:
# notwendige Python-Pakete importieren
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer

# Datensatz vorbereiten
cols = ['sentiment', 'id', 'date','query_string', 'user', 'text']
df = pd.read_csv("training.1600000.processed.noemoticon.csv",encoding = "ISO-8859-1",header=None, names=cols)
df.drop(['id','date','query_string','user'],axis=1,inplace=True)
df['sentiment'] = df['sentiment'].map({0: 0, 4: 1})

# Hier wird eine Methode zur Datenvorverarbeitung definiert
# Zuerst muss eine Liste aller zusammengezogenen Wörter in der englischen Sprache erstellt
con_dict = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", 
            "weren't":"were not","haven't":"have not","hasn't":"has not",
            "hadn't":"had not","won't":"will not","wouldn't":"would not",
            "don't":"do not", "doesn't":"does not","didn't":"did not",
            "can't":"can not","couldn't":"could not","shouldn't":"should not",
            "mightn't":"might not","mustn't":"must not", "'s":" is", "'re":" are", 
            "'m":" am", "'ll":" will", "'d":" would"}

con_re = re.compile('(%s)' % '|'.join(con_dict.keys()))
tok = WordPunctTokenizer()


# Hier findet die Datenvorverarbeitung statt.

def dvv(text):    
    wort_trennen = con_re.sub(lambda x: con_dict[x.group()],str(text))
    url_entfernen = re.sub('((www.[^ ]+)|(https?://[^ ]+))','',wort_trennen)
    soup = BeautifulSoup(url_entfernen, 'lxml')
    souped = soup.get_text()
    klein_umwandeln = str(souped).lower()
    zahlen_entfern = re.sub(r'((@[A-Za-z0-9_]+)|([0-9]+)|([&?.!/;:%*+-<>§$]))', " ", klein_umwandeln)
    wort = [x for x  in tok.tokenize(zahlen_entfern) if len(x) > 1]
    return (" ".join(wort)).strip()

# Alle Texte vom Datensatz werden verarbeitet...
dvv_text = []
for i in range(0,len(df)):
    dvv_text.append(dvv(df['text'][i]))
print('Daten sind fertig verarbeitet')

#...und als eine .csv-Datei abgespeichert
dvv_df = pd.DataFrame(dvv_text,columns=['text'])
dvv_df.text.dropna(inplace=True)
dvv_df['sentiment'] = df['sentiment']
dvv_df.to_csv('clean_tweet.csv',encoding='utf-8')

Daten sind fertig verarbeitet


In [2]:
'''
 Da es untersucht werden sollte, ob die Entfernung von Stoppwörter notwendig ist, 
 werden zwei unterschiedliche verarbeitete Datensätze abgespeichert.
 Dieser enthält keine Stoppwörter.
'''
import nltk
from nltk.corpus import stopwords
STOPWORDS = stopwords.words('english')
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
dvv_df['text'] = dvv_df['text'].apply(lambda text: cleaning_stopwords(text))
dvv_df.to_csv('clean_tweet_sw.csv',encoding='utf-8')